In [35]:
import json
import pandas as pd
import numpy as np

price_list = pd.read_json("price_list.json")
food_transactions = pd.read_csv("food_transactions.csv")
price_list

,price
Big_combo,169
Fam_combo,599
S_Burger,89
S_Drink,49
S_Fries,69
Snack_combo,99


In [2]:
food_transactions

,Unnamed: 0,date,branch,order
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink"
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink"
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink"
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink"
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink"
...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink"
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries"
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries"
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink"


### Prompt:
You are an analyst for a fastfood restaurant. The company has tasked you to analyze their past transactions. Each row represents a single transaction/order from one of their ten (10) branches. The order is recorded as a __`string`__ which is formatted as: 

"`[quantity1][item_code1], [quantity2][item_code2], [quantity3][item_code3]`"

However, because of some technical error, the __number of combo meals purchased__ as well as the __total order value__ was lost, so we'll need to restore that.

### You are tasked to come up with the following:
1. Given the order column, count how many of each product was purchased
2. Given each product count, get the number of supposed combo meals purchased. <br><i>(You may update the product count when the combo meals are counted)</i>.
3. Calculate the total order value.
4. Determine which branch made the most revenue, and how much?

### The combo meals are detailed below:
1. Family combo: 4 burgers, 4 fries, 4 drinks
2. Big combo: 1 burgers, 1 fries, 1 drinks
3. Snack combo: 1 fries, 1 drinks

In [34]:
# 1. Given the order column, count how many of each product was purchased
def count(order):
    Burgers = 0
    Fries = 0
    Drinks = 0
    order = str(order)
    separate_items = order.split(", ")
    for item in separate_items:
        menu_item = str(item[3])
        count = int(item[0])
        if menu_item == "B":
            Burgers+=count
            continue
        elif menu_item == "F":
            Fries+=count
            continue
        elif menu_item == "D":
            Drinks+=count
            continue
    return "Burgers:"+str(Burgers)+ " Fries:"+str(Fries)+" Drinks:"+str(Drinks)


food_transactions
count("1S_Burger, 4S_Fries, 5S_Drink")

'Burgers:1 Fries:4 Drinks:5'

In [36]:
food_transactions["Burgers"] = food_transactions['order'].apply(lambda x: str(x)[str(x).find("S_Burger")-1] if "S_Burger" in str(x) else 0)
food_transactions["Fries"] = food_transactions['order'].apply(lambda x: str(x)[str(x).find("S_Fries")-1] if "S_Fries" in str(x) else 0)
food_transactions["Drinks"] = food_transactions['order'].apply(lambda x: str(x)[str(x).find("S_Drink")-1] if "S_Drink" in str(x) else 0)
food_transactions

,Unnamed: 0,date,branch,order,Burgers,Fries,Drinks
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink",1,4,5
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink",2,4,3
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink",2,4,2
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink",0,6,1
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink",5,0,6
...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink",0,3,6
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries",7,6,0
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries",4,6,0
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink",7,6,5


In [42]:
# 2. Given each product count, get the number of supposed combo meals purchased.
# (You may update the product count when the combo meals are counted).
# The combo meals are detailed below:
# Family combo: 4 burgers, 4 fries, 4 drinks
# Big combo: 1 burgers, 1 fries, 1 drinks
# Snack combo: 1 fries, 1 drinks

for i in range(len(food_transactions)):
    if int(food_transactions["Burgers"][i]) == 4 and int(food_transactions["Fries"][i]) == 4 and int(food_transactions["Drinks"][i]) == 4:
        print("Family Combo")
    elif int(food_transactions["Burgers"][i]) == 1 and int(food_transactions["Fries"][i]) == 1 and int(food_transactions["Drinks"][i]) == 1:
        print("Big Combo")
    elif int(food_transactions["Fries"][i]) == 1 and int(food_transactions["Drinks"][i]) == 1:
        print("Snack Combo")
    else:
        print("Nope")
        
food_transactions

Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Snack Combo
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Snack Combo
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Snack Combo
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope
Nope

,Unnamed: 0,date,branch,order,Burgers,Fries,Drinks
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink",1,4,5
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink",2,4,3
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink",2,4,2
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink",0,6,1
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink",5,0,6
...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink",0,3,6
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries",7,6,0
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries",4,6,0
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink",7,6,5


In [43]:
# 3. Calculate the total order value.

food_transactions["Order Value"] = []

for i in range(len(food_transactions)):
    if food_transactions["family combo"][i] == "Family Combo":
        food_transactions["total order value"].append(1 * price_list["price"].iloc[1])
    elif food_transactions["big combo"][i] == "Big Combo":
        food_transactions["total order value"].append(1 * price_list["price"].iloc[0])
    elif food_transactions["snack combo"][i] == "Snack Combo":
        food_transactions["total order value"].append(1 * price_list["price"].iloc[5])
    else:
        sum = (food_transactions["burger"][i] * price_list["price"].iloc[2]) + (food_transactions["drink"][i] * price_list["price"].iloc[3]) + (food_transactions["fries"][i] * price_list["price"].iloc[4])
        food_transactions["total order value"].append(sum)


ValueError: Length of values (0) does not match length of index (134775)

In [44]:
# 4. Determine which branch made the most revenue, and how much?

food_transactions.groupby("branch").sum(numeric_only = True)["Order Value"].sort_values(ascending=False)

KeyError: 'Order Value'